{"cell_type":"markdown","metadata":{},"source":[
   "# 🤖 Fundamentos de LLMs y Prompting\n",
   "\n",
   "Objetivo: aprender a usar LLMs (OpenAI GPT, Google Gemini) para tareas de ingeniería de datos mediante técnicas de prompting efectivas.\n",
   "\n",
   "- Duración: 90-120 min\n",
   "- Dificultad: Media\n",
   "- APIs: OpenAI GPT-4/3.5, Google Gemini Pro\n"
  ]},
  {"cell_type":"markdown","metadata":{},"source":["## 1. Setup y primera llamada"]},
  {"cell_type":"code","metadata":{},"execution_count":null,"outputs":[],"source":[
   "# pip install openai google-generativeai python-dotenv\n",
   "import os\n",
   "from openai import OpenAI\n",
   "import google.generativeai as genai\n",
   "\n",
   "# Configurar APIs\n",
   "client_openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))\n",
   "genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))\n",
   "\n",
   "print('✅ APIs configuradas')\n"
  ]},
  {"cell_type":"markdown","metadata":{},"source":["## 2. Primera consulta - OpenAI"]},
  {"cell_type":"code","metadata":{},"execution_count":null,"outputs":[],"source":[
   "def ask_openai(prompt: str, model='gpt-3.5-turbo') -> str:\n",
   "    \"\"\"Consulta a OpenAI.\"\"\"\n",
   "    response = client_openai.chat.completions.create(\n",
   "        model=model,\n",
   "        messages=[{'role': 'user', 'content': prompt}],\n",
   "        temperature=0.7\n",
   "    )\n",
   "    return response.choices[0].message.content\n",
   "\n",
   "pregunta = 'Explica en 2 frases qué es un Data Lake'\n",
   "respuesta = ask_openai(pregunta)\n",
   "print(f'OpenAI: {respuesta}')\n"
  ]},
  {"cell_type":"markdown","metadata":{},"source":["## 3. Primera consulta - Google Gemini"]},
  {"cell_type":"code","metadata":{},"execution_count":null,"outputs":[],"source":[
   "def ask_gemini(prompt: str, model='gemini-1.5-flash') -> str:\n",
   "    \"\"\"Consulta a Google Gemini.\"\"\"\n",
   "    model_gemini = genai.GenerativeModel(model)\n",
   "    response = model_gemini.generate_content(prompt)\n",
   "    return response.text\n",
   "\n",
   "respuesta_gemini = ask_gemini(pregunta)\n",
   "print(f'Gemini: {respuesta_gemini}')\n"
  ]},
  {"cell_type":"markdown","metadata":{},"source":["## 4. Comparación de respuestas"]},
  {"cell_type":"code","metadata":{},"execution_count":null,"outputs":[],"source":[
   "# Comparar ambos modelos\n",
   "test_prompts = [\n",
   "    '¿Qué es ETL?',\n",
   "    'Diferencia entre Data Lake y Data Warehouse',\n",
   "    'Ventajas de usar Apache Airflow'\n",
   "]\n",
   "\n",
   "for prompt in test_prompts:\n",
   "    print(f'\\n❓ {prompt}')\n",
   "    print(f'OpenAI: {ask_openai(prompt)[:100]}...')\n",
   "    print(f'Gemini:  {ask_gemini(prompt)[:100]}...')\n"
  ]},"

## 0. Configuración y dependencias

In [ ]:
# pip install openai python-dotenv tiktoken
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    print('⚠️ OPENAI_API_KEY no configurada. Define en .env o variable de entorno.')
else:
    print('✅ API Key cargada')

## 1. Primera llamada: completar texto

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'system', 'content': 'Eres un asistente experto en ingeniería de datos.'},
        {'role': 'user', 'content': '¿Qué es un data lakehouse en 2 líneas?'}
    ],
    temperature=0.7,
    max_tokens=150
)

print(response.choices[0].message.content)
print(f"\n💰 Tokens usados: {response.usage.total_tokens}")

## 2. Técnicas de prompting

### 2.1 Zero-shot: sin ejemplos

In [ ]:
def ask_llm(prompt: str, model='gpt-3.5-turbo', temp=0.3):
    resp = client.chat.completions.create(
        model=model,
        messages=[{'role':'user','content':prompt}],
        temperature=temp,
        max_tokens=300
    )
    return resp.choices[0].message.content

result = ask_llm('Clasifica este error como: database, network, o application:\nError: Connection timeout after 30s')
print(result)

### 2.2 Few-shot: con ejemplos

In [ ]:
few_shot_prompt = '''
Clasifica el tipo de error (database, network, application).

Ejemplos:
Error: Connection timeout after 30s → network
Error: Table 'users' does not exist → database
Error: NullPointerException in transform.py → application

Ahora clasifica:
Error: Permission denied on SELECT query
'''
print(ask_llm(few_shot_prompt, temp=0))

### 2.3 Chain-of-Thought (CoT): razonamiento paso a paso

In [ ]:
cot_prompt = '''
Tengo una tabla 'ventas' con 10 millones de filas. Una consulta tarda 45 segundos.
La consulta filtra por fecha (últimos 7 días) y cliente_id, y hace GROUP BY producto_id.

Analiza paso a paso por qué es lenta y sugiere 3 optimizaciones concretas.
'''
print(ask_llm(cot_prompt, temp=0.2))

## 3. Uso práctico: generación de documentación

In [ ]:
code_snippet = '''
def extract_sales_data(start_date, end_date):
    conn = psycopg2.connect(DB_URI)
    query = f"SELECT * FROM sales WHERE date >= '{start_date}' AND date <= '{end_date}'"
    df = pd.read_sql(query, conn)
    conn.close()
    return df
'''

doc_prompt = f'''
Genera documentación en formato docstring para esta función Python:

{code_snippet}

Incluye: descripción, parámetros, retorno, ejemplo de uso.
'''
print(ask_llm(doc_prompt, temp=0.1))

## 4. Buenas prácticas

### 4.1 Control de temperatura
- `temperature=0`: determinístico, ideal para clasificación/extracción.
- `temperature=0.7-1.0`: creativo, bueno para generación de ideas.

### 4.2 Límites de tokens
- Estima tokens con tiktoken antes de enviar.
- Usa `max_tokens` para controlar costos.

### 4.3 System prompts
- Define rol y contexto en mensaje `system`.
- Establece formato de salida esperado.

### 4.4 Manejo de errores

In [ ]:
def safe_llm_call(prompt: str, retries=3):
    for i in range(retries):
        try:
            return ask_llm(prompt)
        except Exception as e:
            if i == retries - 1:
                return f'Error after {retries} retries: {e}'
            import time
            time.sleep(2 ** i)
    return None

result = safe_llm_call('Resume en 1 línea qué es Apache Spark.')
print(result)

## 5. Conteo de tokens y estimación de costos

In [ ]:
import tiktoken

def count_tokens(text: str, model='gpt-3.5-turbo') -> int:
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

sample = 'Genera un pipeline ETL en Python que lea CSV, valide con Pandera y escriba a Parquet.'
tokens = count_tokens(sample)
cost_input = tokens * 0.0015 / 1000  # GPT-3.5-turbo input: $0.0015/1K tokens
print(f'Prompt: {tokens} tokens, costo estimado: ${cost_input:.6f}')

## 6. Ejercicios

1. Crea un prompt que genere un schema Pandera desde una descripción en texto.
2. Usa few-shot prompting para clasificar errores de logs en 5 categorías.
3. Implementa una función que resuma un archivo de configuración YAML en lenguaje natural.
4. Genera un prompt que convierta una consulta SQL a una explicación en español.